In [1]:
import sys
sys.path += ["../src"]
import utils
from glob import glob
import matplotlib.pyplot as plt 
import seaborn as sns
from matplotlib.pyplot import subplots as sbp 
from importlib import reload
import jl_vae
import jl_nflows_geo_coordinates_2 as nfg
from jl_nflows_geo_coordinates import load_nf as load_dict

from _51_abm_functions import cod_prov_abbrv_df

# Global Spatial Autocorrelation
from spatial_autocorrelation import get_moransI, moransI_scatterplot, hypothesis_testing
# Local Spatial Autocorrelation
from spatial_autocorrelation import get_localMoransI, LISA_scatterplot
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
from sklearn.preprocessing import OneHotEncoder
from jl_synthetic_pop_all_provinces import add_cat_features
from ipfn import ipfn
import config


In [2]:


features_synpop = ['flag_garage', 'flag_pertinenza', 'flag_air_conditioning',
            'flag_multi_floor', 'log_mq', 'ANNO_COSTRUZIONE_1500_1965',
            'ANNO_COSTRUZIONE_1965_1985', 'ANNO_COSTRUZIONE_1985_2005',
            'ANNO_COSTRUZIONE_2005_2025', 'ANNO_COSTRUZIONE_Missing',
            'High_energy_class', 'Low_energy_class', 'Medium_energy_class',
            'Missing_energy_class', 'COD_CAT_A02', 'COD_CAT_A03',
            'COD_CAT_A_01_07_08', 'COD_CAT_A_04_05', 'floor_0.0', 'floor_1.0',
            'floor_2.0', 'floor_3.0', 'floor_Missing', 'floor_plus_4',
            'flag_air_conditioning_Missing', 'flag_multi_floor_Missing', 'y', 'x','log_price']


def transform_bins(df, vars):
    df1 = df.copy()
    for var in vars:
        df1[var + "_bin"] =  pd.qcut(df1[var], q = 5, labels = False, duplicates = "drop").astype(int)
        df1.drop(columns = [var], inplace = True)
    return df1

def transform_bins_to_real(sample_df, df_real, var):
    bins_values = pd.qcut(df_real[var], q = 5, labels = False, duplicates = "drop", retbins = True)[1]
    np.random.seed(2)
    sample_df[var] = [np.random.uniform(low = bins_values[u], high = bins_values[u + 1]) for u in sample_df[var + "_bin"]]
    sample_df.drop(columns = [var + "_bin"], inplace = True)
    return sample_df

    


In [3]:
geo_dict = jl_vae.load_geo_data()

In [14]:

if True:
    
    date = "250718"

    prov = "AL"
    if True:
 
        print(prov)
        if True:
            df_real = pd.read_csv(f"/data/housing/data/intermediate/jl_pop_synth/real_populations/df_real_{prov}.csv", index_col = 0)[features_synpop + ["CAP"]]
            df_real95 = df_real.sample(frac = 0.95, random_state = 1111)

            sample_locs = True
            np.random.seed(2)
            # matched_df = []
            # while sample_locs:
            #     ran_xy = np.random.uniform(low = df_real["x"].min(), high = df_real["x"].max(), size = 1000000), np.random.uniform(low = df_real["y"].min(), high = df_real["y"].max(), size = 1000000)

            #     matched_df.append(utils.spatial_matching_ABM(pd.DataFrame({"x": ran_xy[0], "y": ran_xy[1]}), geo_dict["hydro_risk"], geo_dict["census"], 
            #                                             geo_dict["omi_og"], geo_dict["cap"]).query("prov_abbrv == @prov"))
            #     df_locations = pd.concat(matched_df)
            #     if len(df_locations["CAP"].unique()) >= len(df_real["CAP"].unique()):
            #         sample_locs = False

            ran_xy = np.random.uniform(low = df_real["x"].min(), high = df_real["x"].max(), size = 1000000), np.random.uniform(low = df_real["y"].min(), high = df_real["y"].max(), size = 1000000)

            df_locations = utils.spatial_matching_ABM(pd.DataFrame({"x": ran_xy[0], "y": ran_xy[1]}), geo_dict["hydro_risk"], geo_dict["census"], 
                                                         geo_dict["omi_og"], geo_dict["cap"]).query("prov_abbrv == @prov")
            
            

            sample_dfs = []
            for df_ in [df_real.copy(), df_real95.copy()]:
                df_ = df_.dropna(subset = "CAP").assign(CAP = lambda x: [f"{int(u):05d}" for u in x["CAP"]])
                df_.drop(columns = ["x", "y"], inplace = True)
            
                df_sample_list = []
            
                for cap in df_["CAP"].unique():
                    locations_cap = df_locations.query("CAP == @cap")
                    df_cap = df_.query("CAP == @cap").dropna()
                    N_cap = len(df_cap)
                    if N_cap > 1:
                        df_cap = transform_bins(df_cap, ["log_mq", "log_price"])            
                        df_counts = df_cap.value_counts().reset_index()
                        sample_df_cap = df_counts.sample(n = 10 * N_cap, weights = df_counts["count"],
                                                    random_state = 2, replace = True).drop(columns = ["count"])

                        sample_df_cap = transform_bins_to_real(sample_df_cap, df_, "log_mq")
                        sample_df_cap = transform_bins_to_real(sample_df_cap, df_, "log_price")
                    else:
                        sample_df_cap = pd.concat([df_cap] * 10)
                    

                    if len(locations_cap) > 0:
                        cap_locs = locations_cap.sample(n = 10 * N_cap, replace = True, random_state = 2)
                        sample_df_cap["x"] = list(cap_locs["GEO_LONGITUDINE_BENE_ROUNDED"])
                        sample_df_cap["y"] = list(cap_locs["GEO_LATITUDINE_BENE_ROUNDED"])
                        sample_df_cap["CAP"] = cap
                    
                    df_sample_list.append(sample_df_cap)
                sample_df = pd.concat(df_sample_list)
                sample_dfs.append(sample_df)

AL


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [12]:
sorted(df_["CAP"].unique())

['13040',
 '15010',
 '15011',
 '15012',
 '15013',
 '15015',
 '15016',
 '15017',
 '15018',
 '15019',
 '15020',
 '15021',
 '15022',
 '15023',
 '15024',
 '15025',
 '15026',
 '15027',
 '15028',
 '15029',
 '15030',
 '15031',
 '15032',
 '15033',
 '15034',
 '15035',
 '15036',
 '15038',
 '15039',
 '15040',
 '15041',
 '15042',
 '15043',
 '15044',
 '15045',
 '15046',
 '15047',
 '15048',
 '15049',
 '15050',
 '15051',
 '15052',
 '15053',
 '15054',
 '15055',
 '15056',
 '15057',
 '15058',
 '15059',
 '15060',
 '15061',
 '15062',
 '15063',
 '15064',
 '15065',
 '15066',
 '15067',
 '15068',
 '15069',
 '15070',
 '15071',
 '15072',
 '15073',
 '15074',
 '15075',
 '15076',
 '15077',
 '15078',
 '15079',
 '15121',
 '15122']

In [13]:
sorted(df_locations["CAP"].unique())

['15010',
 '15011',
 '15012',
 '15013',
 '15015',
 '15016',
 '15017',
 '15018',
 '15019',
 '15020',
 '15021',
 '15022',
 '15023',
 '15024',
 '15025',
 '15026',
 '15027',
 '15028',
 '15029',
 '15030',
 '15031',
 '15032',
 '15033',
 '15034',
 '15035',
 '15036',
 '15038',
 '15039',
 '15040',
 '15041',
 '15042',
 '15043',
 '15044',
 '15045',
 '15046',
 '15047',
 '15048',
 '15049',
 '15050',
 '15051',
 '15052',
 '15053',
 '15054',
 '15055',
 '15056',
 '15057',
 '15058',
 '15059',
 '15060',
 '15061',
 '15062',
 '15063',
 '15064',
 '15065',
 '15066',
 '15067',
 '15068',
 '15069',
 '15070',
 '15071',
 '15072',
 '15073',
 '15074',
 '15075',
 '15076',
 '15077',
 '15078',
 '15079',
 '15121',
 '15122',
 '27020',
 '27030']

In [102]:
df.columns

Index(['flag_garage', 'flag_pertinenza', 'flag_air_conditioning', 'CAP',
       'anno_costruzione', 'log_mq_bin', 'log_price_bin'],
      dtype='object')

In [103]:
def get_ipf_input(df):
    seed = df.groupby(list(df.columns)).size().reset_index(name="count")
    remove_vals = []
    aggregates = []
    dimensions = []
    for var in df.columns:
        agg = df.groupby(var).size()
        aggregates.append(agg)
        dimensions.append([var])
        remove_vals.append([var, list(agg[agg == 1].index)])
    return seed, aggregates, dimensions, remove_vals

In [104]:
def clean_df(df):
    remove_obs = []
    repeat = True
    rep = 0
    df_out = df.copy()
    while repeat:
        rep += 1
        new_removals = []
        for col in df.columns:
            val_counts = df_out[col].value_counts()
            remove_class = list(val_counts[val_counts == 1].index)
            new_removals += list(df_out[df_out[col].isin(remove_class)].index)
        remove_obs += new_removals
        df_out = df_out[~df_out.index.isin(remove_obs)]
        print(len(remove_obs), len(df_out))
        if new_removals == []:
            repeat = False

        if rep > 20:
            repeat = False
    return df_out, remove_obs

In [105]:
CAPS_prov = df_real["CAP"].value_counts()

In [106]:
for _,(cap,count) in CAPS_prov.reset_index().iterrows():
    break

In [112]:
if count > 1:
    # df = transform_bins(df_real.query("CAP == @cap").dropna().drop(columns = ["CAP"]), ["log_mq", "log_price"])
    df = transform_bins(df_real.dropna().drop(columns = ["CAP"]), ["log_mq", "log_price"])
    # cleaned_df, remove_obs = clean_df(df)
    cleaned_df = df.copy()
    seed, aggregates, dimensions, remove_vals = get_ipf_input(cleaned_df)
    model = ipfn.ipfn(original = seed, aggregates = aggregates,
             dimensions = dimensions, weight_col = "count",
             convergence_rate = 1e-5, max_iteration=500)
    result = model.iteration()

In [113]:
result.sort_values("count", ascending = False)

,log_price_bin,flag_garage,flag_pertinenza,flag_air_conditioning,anno_costruzione,log_mq_bin,count
0,0,0.0,0.0,0.0,1500_1965,0,1110
25,0,0.0,0.0,0.0,1965_1985,0,783
411,4,1.0,0.0,0.0,1965_1985,4,671
1,1,0.0,0.0,0.0,1500_1965,0,517
436,4,1.0,0.0,0.0,1985_2005,4,495
...,...,...,...,...,...,...,...
200,4,0.0,0.0,1.0,Missing,3,1
536,3,1.0,0.0,1.0,2005_2025,0,1
199,3,0.0,0.0,1.0,Missing,0,1
108,0,0.0,0.0,0.0,Missing,2,1


In [95]:
cleaned_df.value_counts().reset_index()

,flag_garage,flag_pertinenza,flag_air_conditioning,anno_costruzione,log_mq_bin,log_price_bin,count
0,1.0,0.0,0.0,2005_2025,4,4,56
1,0.0,0.0,0.0,1965_1985,0,0,51
2,0.0,0.0,0.0,1500_1965,0,0,44
3,1.0,0.0,0.0,1965_1985,4,4,33
4,1.0,0.0,0.0,2005_2025,0,0,33
...,...,...,...,...,...,...,...
228,0.0,1.0,0.0,1985_2005,2,2,1
229,0.0,1.0,0.0,1965_1985,4,4,1
230,0.0,1.0,0.0,1965_1985,3,0,1
231,0.0,1.0,0.0,1965_1985,1,0,1


In [65]:
seed, aggregates, dimensions, remove_vals = get_ipf_input(clean_df)

In [ ]:

# weights = result.df

In [ ]:
result["count"]

179

In [ ]:
df = transform_bins(df_real.dropna(), ["log_mq", "log_price"])
seed, aggregates, dimensions, remove_vals = get_ipf_input(df)

[['flag_garage', []],
 ['flag_pertinenza', []],
 ['flag_air_conditioning', []],
 ['CAP', []],
 ['energy_class', []],
 ['COD_CAT', []],
 ['anno_costruzione', []],
 ['log_mq_bin', []],
 ['log_price_bin', []]]

In [170]:
df = transform_bins(df_real, ["log_mq", "log_price"])#.drop(columns = ["CAP"])
# aggregates = [df[var].value_counts().sort_index() for var in df.columns]
# dimensions = list(df.columns)
seed = df.groupby(list(df.columns)).size().reset_index(name="count")
remove_vals = []
aggregates_ = []
dimensions = []
for var in df.columns:
    agg = df.groupby(var).size()
    aggregates_.append(agg)
    dimensions.append([var])
    remove_vals.append([var, list(agg[agg == 1].index)])
# aggregates = [df[col].value_counts().sort_index().values for col in df.columns]
# dimensions = [[col] for col in df.columns]

In [178]:
remove_obs = []
for var, idx in remove_vals:
    if len(idx) > 0:
        remove_obs += list(df[df[var].isin(idx)].index)

In [ ]:
39022

In [206]:
seed.query("CAP == '39022'")

,flag_garage,flag_pertinenza,flag_air_conditioning,CAP,energy_class,COD_CAT,anno_costruzione,log_mq_bin,log_price_bin,count
115,1.0,0.0,0.0,39022,Missing_energy_class,A02,1965_1985,0,0.0,1


In [179]:
remove_obs

[228, 363, 859]

In [162]:
agg = aggregates[3]

In [215]:
df = seed.copy()
steps = len(aggregates)
weight_col = "count"
if True:
        tables = [df]
        for inc in range(steps - 1):
            tables.append(df.copy())
        original = df.copy()

        # Calculate the new weights for each dimension
        inc = 0
        for features in dimensions:
            if inc == (steps - 1):
                table_update = df
                table_current = tables[inc].copy()
            else:
                table_update = tables[inc + 1]
                table_current = tables[inc]

            tmp = table_current.groupby(features)[weight_col].sum()
            xijk = aggregates[inc]

            feat_l = []
            for feature in features:
                feat_l.append(np.unique(table_current[feature]))
            table_update.set_index(features, inplace=True)
            table_current.set_index(features, inplace=True)

            multi_index_flag = isinstance(table_update.index, pd.MultiIndex)
            if multi_index_flag:
                if not table_update.index.is_monotonic_increasing:
                    table_update.sort_index(inplace=True)
                if not table_current.index.is_monotonic_increasing:
                    table_current.sort_index(inplace=True)
            
            for feature in product(*feat_l):
                print(feature)
                break

            for feature in product(*feat_l):
                
                den = tmp.loc[feature]
                # calculate new weight for this iteration

                if not multi_index_flag:
                    msk = table_update.index == feature[0]
                else:
                    msk = feature

                if den == 0:
                    table_update.loc[msk, weight_col] =\
                        table_current.loc[feature, weight_col] *\
                        xijk.loc[feature]
                else:
                    table_update.loc[msk, weight_col] = \
                        table_current.loc[feature, weight_col].astype(float) * \
                        xijk.loc[feature] / den
                    
            table_update.reset_index(inplace=True)
            table_current.reset_index(inplace=True)
            inc += 1
            feat_l = []

        # Calculate the max convergence rate
        max_conv = 0
        inc = 0
        for features in dimensions:
            tmp = table_update.groupby(features)[weight_col].sum()
            ori_ijk = aggregates[inc]
            temp_conv = max(abs(tmp / ori_ijk - 1))
            if temp_conv > max_conv:
                max_conv = temp_conv
            inc += 1

                

(0.0,)
(0.0,)
(0.0,)
('39010',)


AttributeError: 'numpy.float64' object has no attribute 'loc'

In [26]:
table_current.loc[feature]

flag_air_conditioning                     0.0
flag_garage                               1.0
flag_pertinenza                           1.0
energy_class             Missing_energy_class
COD_CAT                                   A02
anno_costruzione                    2005_2025
log_mq_bin                                  3
log_price_bin                             2.0
count                                 3.62419
Name: 39017, dtype: object

In [76]:
model = ipfn.ipfn(original = seed, aggregates = aggregates,
             dimensions=dimensions, weight_col = "count",
             convergence_rate=1e-5, max_iteration=500)
result = model.iteration()
# weights = result.df

AttributeError: 'numpy.float64' object has no attribute 'loc'

In [95]:
df.value_counts().reset_index().assign(count = lambda x: x["count"] / x["count"].sum() * 1568.0)

,flag_garage,flag_pertinenza,flag_air_conditioning,energy_class,COD_CAT,anno_costruzione,x_bin,y_bin,log_mq_bin,log_price_bin,count
0,0.0,0.0,0.0,Missing_energy_class,A02,1500_1965,1,1,0,0.0,27.787342
1,1.0,0.0,0.0,Missing_energy_class,A02,1985_2005,4,4,4,4.0,19.848101
2,1.0,0.0,0.0,Missing_energy_class,A02,2005_2025,4,4,4,4.0,15.878481
3,0.0,0.0,0.0,Missing_energy_class,A02,1985_2005,4,4,0,0.0,11.908861
4,1.0,0.0,0.0,Missing_energy_class,A02,2005_2025,1,2,2,2.0,11.908861
...,...,...,...,...,...,...,...,...,...,...,...
327,0.0,1.0,0.0,Missing_energy_class,A02,1965_1985,0,1,0,0.0,3.969620
328,0.0,1.0,0.0,Missing_energy_class,A02,1500_1965,3,1,3,3.0,3.969620
329,0.0,1.0,0.0,Missing_energy_class,A02,1500_1965,3,1,3,2.0,3.969620
330,0.0,1.0,0.0,Missing_energy_class,A02,1500_1965,2,0,3,1.0,3.969620


In [99]:
result.sample(n = 10, weights = result["count"])

,log_price_bin,flag_garage,flag_pertinenza,flag_air_conditioning,energy_class,COD_CAT,anno_costruzione,x_bin,y_bin,log_mq_bin,count
268,0.0,1.0,0.0,0.0,Missing_energy_class,A02,2005_2025,4,3,0,3.753142
246,4.0,1.0,0.0,0.0,Missing_energy_class,A02,2005_2025,1,2,3,6.099109
16,4.0,0.0,0.0,0.0,Missing_energy_class,A02,1500_1965,2,2,3,3.396339
81,0.0,0.0,0.0,0.0,Missing_energy_class,A02,1985_2005,4,4,0,11.830792
258,2.0,1.0,0.0,0.0,Missing_energy_class,A02,2005_2025,3,2,0,4.199900
58,1.0,0.0,0.0,0.0,Missing_energy_class,A02,1965_1985,3,1,3,4.762042
28,3.0,0.0,0.0,0.0,Missing_energy_class,A02,1965_1985,0,0,4,4.913030
312,2.0,1.0,1.0,0.0,Missing_energy_class,A02,2005_2025,0,3,2,4.196578
208,2.0,1.0,0.0,0.0,Missing_energy_class,A02,1985_2005,2,0,3,3.700037
48,4.0,0.0,0.0,0.0,Missing_energy_class,A02,1965_1985,2,1,4,4.444800


In [ ]:

# 1. Load your df as given

# 2. Create bins & marginals
df['x_bin'] = pd.qcut(df['x'], q = 10, labels=False)
df['y_bin'] = pd.qcut(df['y'], q = 10, labels=False)

agg_x = df['x_bin'].value_counts().sort_index()
agg_y = df['y_bin'].value_counts().sort_index()
agg_bool1 = df['bool_1'].value_counts().sort_index()
# [... repeat for other variables ...]

aggregates = [agg_x.values, agg_y.values, agg_bool1.values, ...]
dimensions = [['x_bin'], ['y_bin'], ['bool_1'], ...]

# 3. Run IPF
orig = df[['x_bin','y_bin','bool_1', ...]]  # only constraint columns
ipf = ipfn(orig, aggregates, dimensions)
res = ipf.iteration(max_iteration=500, convergence_rate=1e-5)
weights = res.df

# 4. Weighted sampling
weights['prob'] = weights['weight'] / weights['weight'].sum()
synthetic_df = df.sample(
    n=len(df),
    replace=True,
    weights=weights['prob']
).reset_index(drop=True)


NameError: name 'df' is not defined

In [1]:
import sys
sys.path += ["../src"]
import utils
from glob import glob
import matplotlib.pyplot as plt 
import seaborn as sns
from matplotlib.pyplot import subplots as sbp 
from importlib import reload
import jl_vae
import jl_nflows_geo_coordinates_2 as nfg
from jl_nflows_geo_coordinates import load_nf as load_dict

from _51_abm_functions import cod_prov_abbrv_df

# Global Spatial Autocorrelation
from spatial_autocorrelation import get_moransI, moransI_scatterplot, hypothesis_testing
# Local Spatial Autocorrelation
from spatial_autocorrelation import get_localMoransI, LISA_scatterplot
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
from sklearn.preprocessing import OneHotEncoder
from jl_synthetic_pop_all_provinces import add_cat_features
from ipfn import ipfn
import config


In [2]:

def transform_bins(df, vars):
    df1 = df.copy()
    for var in vars:
        df1[var + "_bin"] =  pd.qcut(df1[var], q = 5, labels = False, duplicates = "drop").astype(int)
        df1.drop(columns = [var], inplace = True)
    return df1

def transform_bins_to_real(sample_df, df_real, var):
    bins_values = pd.qcut(df_real[var], q = 5, labels = False, duplicates = "drop", retbins = True)[1]
    np.random.seed(2)
    sample_df[var] = [np.random.uniform(low = bins_values[u], high = bins_values[u + 1]) for u in sample_df[var + "_bin"]]
    sample_df.drop(columns = [var + "_bin"], inplace = True)
    return sample_df

    


In [18]:

if True:
    # geo_dict = jl_vae.load_geo_data()
    date = "250717"

    for _, (prov, cod_prov) in cod_prov_abbrv_df.sort_values("prov_abbrv").iterrows():
        print(prov)
        real_pops = jl_vae.path_pop_synth + f"pop_samples/pop_real_with_hedonic_price"
        df_real_ = pd.read_csv(real_pops + f"/pop_real_full_250110{prov}.csv", index_col = 0)
        df_real_ = add_cat_features(df_real_)
        df_real = df_real_.assign(log_price = lambda x: x["price_corrected"], CAP = lambda x: x["CAP"].astype(str))

        np.random.seed(2)
        # ran_xy = np.random.uniform(low = df_real["x"].min(), high = df_real["x"].max(), size = 1000000), np.random.uniform(low = df_real["y"].min(), high = df_real["y"].max(), size = 1000000)

        # df_locations = utils.spatial_matching_ABM(pd.DataFrame({"x": ran_xy[0], "y": ran_xy[1]}), geo_dict["hydro_risk"], geo_dict["census"], 
        #                                           geo_dict["omi_og"], geo_dict["cap"]).query("prov_abbrv == @prov")
        
        df_real = df_real_.assign(log_price = lambda x: x["price_corrected"], CAP = lambda x: x["CAP"].astype(str))
        df_real = df_real[['flag_garage', 'flag_pertinenza', 'flag_air_conditioning',
            'flag_multi_floor', 
            'log_mq', 'log_price',
            'flag_air_conditioning_Missing', 
            # 'x', 'y', 
            'CAP',
            'energy_class', 
            'COD_CAT',
            'anno_costruzione']]
        
        df_sample_list = []

        for cap in df_real["CAP"].unique():
            df_cap = df_real.query("CAP == @cap").dropna()
            N_cap = len(df_cap)
            if N_cap > 1:
                df_cap = transform_bins(df_cap, ["log_mq", "log_price"])            
                df_counts = df_cap.value_counts().reset_index()
                
                sample_df_cap = df_counts.sample(n = 10 * N_cap, weights = df_counts["count"],
                                            random_state = 2, replace = True).drop(columns = ["count"])

                sample_df_cap = transform_bins_to_real(sample_df_cap, df_real, "log_mq")
                sample_df_cap = transform_bins_to_real(sample_df_cap, df_real, "log_price")
            else:
                sample_df_cap = pd.concat([df_cap] * 10)
            cap_locs = df_locations.query("CAP == @cap").sample(n = 10 * N_cap, random_state = 2)
            sample_df_cap["x"] = list(cap_locs["GEO_LONGITUDINE_BENE_ROUNDED"])
            sample_df_cap["y"] = list(cap_locs["GEO_LATITUDINE_BENE_ROUNDED"])
            sample_df_cap["CAP"] = cap    

            df_sample_list.append(sample_df_cap)
        sample_df = pd.concat(df_sample_list)
        # sample_df.to_csv(jl_vae.path_pop_synth + f"ipf_samples/pop_sample_{prov}_{date}.csv")
        break


        

            






AG


In [19]:
cap_locs["GEO_LONGITUDINE_BENE_ROUNDED"]

583391    13.423687
552259    13.405586
223080    13.397234
893367    13.404734
113833    13.428923
851664    13.439096
948770    13.436452
463187    13.445212
280363    13.403259
684179    13.436004
Name: GEO_LONGITUDINE_BENE_ROUNDED, dtype: float64

In [21]:
sample_df

,flag_garage,flag_pertinenza,flag_air_conditioning,flag_multi_floor,flag_air_conditioning_Missing,CAP,energy_class,COD_CAT,anno_costruzione,log_mq,log_price,x,y
9,0.0,0.0,0.0,0.0,1.0,92024,Missing_energy_class,A03,1965_1985,5.102073,99942.864176,13.863145,37.388755
0,0.0,0.0,0.0,0.0,0.0,92024,Medium_energy_class,A03,1965_1985,5.247497,117383.634742,13.806391,37.351828
12,0.0,0.0,0.0,0.0,0.0,92024,Missing_energy_class,A03,1965_1985,4.282649,69292.524025,13.849301,37.367490
9,0.0,0.0,0.0,0.0,1.0,92024,Missing_energy_class,A03,1965_1985,5.101931,99928.901606,13.893116,37.349320
9,0.0,0.0,0.0,0.0,1.0,92024,Missing_energy_class,A03,1965_1985,5.098795,99618.416010,13.823484,37.348958
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,0.0,1.0,0.0,0.0,0.0,92012,Medium_energy_class,A02,1965_1985,5.192957,41011.067251,13.439096,37.499093
1050,0.0,1.0,0.0,0.0,0.0,92012,Medium_energy_class,A02,1965_1985,5.192957,41011.067251,13.436452,37.465001
1050,0.0,1.0,0.0,0.0,0.0,92012,Medium_energy_class,A02,1965_1985,5.192957,41011.067251,13.445212,37.483883
1050,0.0,1.0,0.0,0.0,0.0,92012,Medium_energy_class,A02,1965_1985,5.192957,41011.067251,13.403259,37.492788


In [5]:
cap_locs[["GEO_LONGITUDINE_BENE_ROUNDED", "GEO_LATITUDINE_BENE_ROUNDED"]]

,GEO_LONGITUDINE_BENE_ROUNDED,GEO_LATITUDINE_BENE_ROUNDED
244296,13.941484,37.233436
676820,13.879161,37.217698
402910,13.942241,37.221549


In [7]:
sample_df_cap#[["x", "y"]]

,flag_garage,flag_pertinenza,flag_air_conditioning,flag_multi_floor,log_mq,log_price,flag_air_conditioning_Missing,CAP,energy_class,COD_CAT,anno_costruzione
1050,0.0,1.0,0.0,0.0,5.192957,41011.067251,0.0,92012,Medium_energy_class,A02,1965_1985
1050,0.0,1.0,0.0,0.0,5.192957,41011.067251,0.0,92012,Medium_energy_class,A02,1965_1985
1050,0.0,1.0,0.0,0.0,5.192957,41011.067251,0.0,92012,Medium_energy_class,A02,1965_1985
1050,0.0,1.0,0.0,0.0,5.192957,41011.067251,0.0,92012,Medium_energy_class,A02,1965_1985
1050,0.0,1.0,0.0,0.0,5.192957,41011.067251,0.0,92012,Medium_energy_class,A02,1965_1985
1050,0.0,1.0,0.0,0.0,5.192957,41011.067251,0.0,92012,Medium_energy_class,A02,1965_1985
1050,0.0,1.0,0.0,0.0,5.192957,41011.067251,0.0,92012,Medium_energy_class,A02,1965_1985
1050,0.0,1.0,0.0,0.0,5.192957,41011.067251,0.0,92012,Medium_energy_class,A02,1965_1985
1050,0.0,1.0,0.0,0.0,5.192957,41011.067251,0.0,92012,Medium_energy_class,A02,1965_1985
1050,0.0,1.0,0.0,0.0,5.192957,41011.067251,0.0,92012,Medium_energy_class,A02,1965_1985


In [9]:
sample_df

,flag_garage,flag_pertinenza,flag_air_conditioning,flag_multi_floor,flag_air_conditioning_Missing,CAP,energy_class,COD_CAT,anno_costruzione,x,y,log_mq,log_price
9,0.0,0.0,0.0,0.0,1.0,92024,Missing_energy_class,A03,1965_1985,NaN,NaN,5.102073,99942.864176
0,0.0,0.0,0.0,0.0,0.0,92024,Medium_energy_class,A03,1965_1985,NaN,NaN,5.247497,117383.634742
12,0.0,0.0,0.0,0.0,0.0,92024,Missing_energy_class,A03,1965_1985,NaN,NaN,4.282649,69292.524025
9,0.0,0.0,0.0,0.0,1.0,92024,Missing_energy_class,A03,1965_1985,NaN,NaN,5.101931,99928.901606
9,0.0,0.0,0.0,0.0,1.0,92024,Missing_energy_class,A03,1965_1985,NaN,NaN,5.098795,99618.416010
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050,0.0,1.0,0.0,0.0,0.0,92012,Medium_energy_class,A02,1965_1985,NaN,NaN,5.192957,41011.067251
1050,0.0,1.0,0.0,0.0,0.0,92012,Medium_energy_class,A02,1965_1985,NaN,NaN,5.192957,41011.067251
1050,0.0,1.0,0.0,0.0,0.0,92012,Medium_energy_class,A02,1965_1985,NaN,NaN,5.192957,41011.067251
1050,0.0,1.0,0.0,0.0,0.0,92012,Medium_energy_class,A02,1965_1985,NaN,NaN,5.192957,41011.067251


In [ ]:

if True:
    # geo_dict = jl_vae.load_geo_data()
    date = "250717"

    for _, (prov, cod_prov) in cod_prov_abbrv_df.sort_values("prov_abbrv").iterrows():
        print(prov)
        real_pops = jl_vae.path_pop_synth + f"pop_samples/pop_real_with_hedonic_price"
        df_real_ = pd.read_csv(real_pops + f"/pop_real_full_250110{prov}.csv", index_col = 0)
        df_real_ = add_cat_features(df_real_)
        df_real = df_real_.assign(log_price = lambda x: x["price_corrected"], CAP = lambda x: x["CAP"].astype(str))

        np.random.seed(2)
        # ran_xy = np.random.uniform(low = df_real["x"].min(), high = df_real["x"].max(), size = 1000000), np.random.uniform(low = df_real["y"].min(), high = df_real["y"].max(), size = 1000000)

        # df_locations = utils.spatial_matching_ABM(pd.DataFrame({"x": ran_xy[0], "y": ran_xy[1]}), geo_dict["hydro_risk"], geo_dict["census"], 
        #                                           geo_dict["omi_og"], geo_dict["cap"]).query("prov_abbrv == @prov")
        
        df_real = df_real_.assign(log_price = lambda x: x["price_corrected"], CAP = lambda x: x["CAP"].astype(str))
        df_real = df_real[['flag_garage', 'flag_pertinenza', 'flag_air_conditioning',
            'flag_multi_floor', 
            'log_mq', 'log_price',
            'flag_air_conditioning_Missing', 
            # 'x', 'y', 
            'CAP',
            'energy_class', 
            'COD_CAT',
            'anno_costruzione']]
        
        df_sample_list = []

        for cap in df_real["CAP"].unique():
            df_cap = df_real.query("CAP == @cap").dropna()
            N_cap = len(df_cap)
            if N_cap > 1:
                df_cap = transform_bins(df_cap, ["log_mq", "log_price"])            
                df_counts = df_cap.value_counts().reset_index()
                
                sample_df_cap = df_counts.sample(n = 10 * N_cap, weights = df_counts["count"],
                                            random_state = 2, replace = True).drop(columns = ["count"])

                sample_df_cap = transform_bins_to_real(sample_df_cap, df_real, "log_mq")
                sample_df_cap = transform_bins_to_real(sample_df_cap, df_real, "log_price")
            else:
                sample_df_cap = pd.concat([df_cap] * 10)
            cap_locs = df_locations.query("CAP == @cap").sample(n = 10 * N_cap, random_state = 2)
            sample_df_cap["x"] = list(cap_locs["GEO_LONGITUDINE_BENE_ROUNDED"])
            sample_df_cap["y"] = list(cap_locs["GEO_LATITUDINE_BENE_ROUNDED"])
            sample_df_cap["CAP"] = cap    

            df_sample_list.append(sample_df_cap)
        sample_df = pd.concat(df_sample_list)
        # sample_df.to_csv(jl_vae.path_pop_synth + f"ipf_samples/pop_sample_{prov}_{date}.csv")
        break


In [25]:
df_cap = df_real.query("CAP == '92024'").dropna()

In [26]:
df_cap

,flag_garage,flag_pertinenza,flag_air_conditioning,flag_multi_floor,log_mq,log_price,flag_air_conditioning_Missing,CAP,energy_class,COD_CAT,anno_costruzione
89,0.0,0.0,0.0,0.0,5.030438,75305.498284,0.0,92024,Low_energy_class,A02,1985_2005
149,0.0,0.0,0.0,0.0,5.159055,87393.554965,0.0,92024,Low_energy_class,A_01_07_08,1985_2005
155,0.0,0.0,0.0,0.0,5.247024,120287.956211,0.0,92024,Missing_energy_class,A_01_07_08,1985_2005
161,0.0,1.0,0.0,0.0,4.770685,68820.436141,0.0,92024,Low_energy_class,A03,1965_1985
198,0.0,0.0,0.0,0.0,4.962845,77078.239136,0.0,92024,Medium_energy_class,A03,1500_1965
202,0.0,0.0,0.0,0.0,4.867535,83166.197922,0.0,92024,Medium_energy_class,A02,1985_2005
225,0.0,0.0,0.0,0.0,4.543295,66633.473374,1.0,92024,Low_energy_class,A02,2005_2025
234,0.0,0.0,0.0,0.0,5.209486,98091.112822,0.0,92024,Medium_energy_class,A03,1965_1985
238,0.0,0.0,0.0,0.0,5.476463,96659.625168,0.0,92024,Medium_energy_class,A03,1965_1985
253,0.0,0.0,0.0,0.0,5.043425,81562.047653,0.0,92024,Low_energy_class,A03,1965_1985


In [ ]:
df_sample = pd.DataFrame

In [ ]:
geo_dict = jl_vae.load_geo_data()


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f546b835110>>
Traceback (most recent call last):
  File "/home/jlenti/miniconda3/envs/synpop/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [16]:
if True:
    date = "250718"

    # for _, (prov, cod_prov) in cod_prov_abbrv_df.sort_values("prov_abbrv").iterrows():
    prov = "CA"
    if True:
        print(prov)
        if True:
            df_real = pd.read_csv(f"/data/housing/data/intermediate/jl_pop_synth/real_populations/df_real_{prov}.csv", index_col = 0)#[features_synpop + ["CAP"]]
            df_real95 = df_real.sample(frac = 0.95, random_state = 1111)

            sample_locs = True
            np.random.seed(2)
            matched_df = []
            k = 0
            while sample_locs:
                print(k)
                ran_xy = np.random.uniform(low = df_real["x"].min(), high = df_real["x"].max(), size = 1000000), np.random.uniform(low = df_real["y"].min(), high = df_real["y"].max(), size = 1000000)

                matched_df.append(utils.spatial_matching_ABM(pd.DataFrame({"x": ran_xy[0], "y": ran_xy[1]}), geo_dict["hydro_risk"], geo_dict["census"], 
                                                        geo_dict["omi_og"], geo_dict["cap"]).query("prov_abbrv == @prov"))
                df_locations = pd.concat(matched_df)
                if len(df_locations["CAP"].unique()) == len(df_real["CAP"].unique()):
                    sample_locs = False
                print(len(df_locations["CAP"].unique()))  
                print(len(df_real["CAP"].unique()))
                k += 1


CA
0
51
46
1


KeyboardInterrupt: 

In [17]:
df_real[~df_real["CAP"].astype(str).isin(df_locations["CAP"].astype(str).unique())]

,flag_garage,flag_pertinenza,flag_air_conditioning,flag_multi_floor,y,x,log_mq,ANNO_COSTRUZIONE_1500_1965,ANNO_COSTRUZIONE_1965_1985,ANNO_COSTRUZIONE_1985_2005,...,floor_Missing,floor_plus_4,log_price,flag_air_conditioning_Missing,flag_multi_floor_Missing,SEZ2011,PRO_COM,CAP,OMI_id,prov_abbrv
0,1.0,0.0,0.0,0.0,39.221,9.122,5.327876,1.0,0.0,0.0,...,1.0,0.0,12.923912,1.0,1.0,9.200900e+11,92009.0,9128.0,B354_B2,CA
1,0.0,0.0,0.0,0.0,39.145,9.017,5.537334,0.0,1.0,0.0,...,1.0,0.0,11.976659,1.0,1.0,9.201100e+11,92011.0,9012.0,B675_E1,CA
2,0.0,0.0,0.0,0.0,39.234,9.124,4.672829,0.0,1.0,0.0,...,1.0,0.0,11.938193,1.0,1.0,9.200900e+11,92009.0,9131.0,B354_C2,CA
3,0.0,0.0,0.0,0.0,39.236,9.131,4.634729,0.0,1.0,0.0,...,1.0,0.0,11.512925,1.0,1.0,9.200900e+11,92009.0,9131.0,B354_D13,CA
4,0.0,0.0,0.0,0.0,39.221,9.125,5.036953,0.0,0.0,1.0,...,1.0,0.0,12.487485,1.0,1.0,9.200900e+11,92009.0,9128.0,B354_B2,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6365,0.0,0.0,0.0,0.0,39.221,9.121,4.852030,1.0,0.0,0.0,...,1.0,0.0,12.345835,1.0,1.0,9.200900e+11,92009.0,9127.0,B354_B2,CA
6366,1.0,1.0,0.0,0.0,39.233,9.179,4.276666,0.0,0.0,0.0,...,1.0,0.0,11.835009,1.0,1.0,9.205100e+11,92051.0,9045.0,H118_D1,CA
6367,1.0,0.0,0.0,0.0,39.007,9.001,4.488636,0.0,0.0,0.0,...,1.0,0.0,12.206073,0.0,1.0,9.205000e+11,92050.0,9050.0,H088_B1,CA
6368,0.0,0.0,1.0,0.0,39.310,8.967,4.875197,0.0,0.0,0.0,...,1.0,0.0,11.849398,0.0,1.0,9.201500e+11,92015.0,9033.0,D259_B1,CA


In [24]:
df_real.dropna(subset = "CAP").assign(CAP = lambda x: [f"{int(u):05d}" for u in x["CAP"]])["CAP"].unique()

array(['09128', '09012', '09131', '09049', '09045', '09047', '09032',
       '09042', '09124', '09134', '09048', '09122', '09126', '09044',
       '09067', '09041', '09121', '09125', '09050', '09020', '09040',
       '09033', '09060', '09129', '09028', '09018', '09127', '09123',
       '09069', '09026', '09068', '09043', '09019', '09010', '09059',
       '09023', '09024', '09034', '09057', '09056', '09061', '09052',
       '09058', '09065', '09063'], dtype=object)

In [20]:
df_locations["CAP"].unique()

array(['09056', '09010', '09061', '09040', '09126', '09020', '09050',
       '09026', '09048', '09069', '09019', '09060', '09023', '09059',
       '09012', '09045', '09058', '09028', '09052', '09041', '09047',
       '09032', '09123', '09051', '09068', '09062', '09034', '09018',
       '09044', '09053', '09049', '09066', '09043', '09057', '09055',
       '09024', '09063', '09065', '09122', '09067', '09134', '09121',
       '09128', '09033', '09125', '09131', '09129', nan, '09042', '09124',
       '09127'], dtype=object)

In [30]:
prov = "LU"
pd.read_csv(f"/data/housing/data/intermediate/jl_pop_synth/real_populations/df_real_{prov}.csv", index_col = 0)["CAP"].unique()

array([55060, 55045, 55054, 55041, 55100, 55049, 55042, 55062, 55012,
       55016, 55047, 55036, 55011, 55025, 55013, 55051, 55023, 55061,
       55027, 55015, 55032, 55022, 55018, 55064, 55014, 55010, 55019,
       51013, 55040, 55031, 55039, 55038, 55020, 55033, 55030, 55035])